# 學生憂鬱症風險分析筆記本
## 版本：v1.0.0
## 日期：2025-05-11

In [ ]:
# 導入必要套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 設定圖表風格
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. 資料載入與檢視

In [ ]:
# 載入資料
df = pd.read_csv('../data/student_depression_dataset.csv')

# 資料基本資訊
print("資料維度：", df.shape)
print("\n資料欄位：")
print(df.info())
print("\n前5筆資料：")
df.head()

## 2. 基礎統計分析

In [ ]:
# 憂鬱症整體比例
depression_rate = df['Depression'].mean() * 100
print(f"整體憂鬱症比例：{depression_rate:.1f}%")

# 學歷分布
degree_counts = df['Degree'].value_counts()
print("\n學歷分布：")
print(degree_counts)

# 學歷與憂鬱症關係
degree_depression = df.groupby('Degree')['Depression'].agg(['count', 'mean']).round(3)
degree_depression.columns = ['總人數', '憂鬱比例']
degree_depression['憂鬱百分比'] = (degree_depression['憂鬱比例'] * 100).round(1)
print("\n學歷憂鬱症比例：")
print(degree_depression)

## 3. 資料視覺化

In [ ]:
# 學歷憂鬱率長條圖
plt.figure(figsize=(10, 6))
degrees = degree_depression.index
depression_rates = degree_depression['憂鬱百分比']
plt.bar(degrees, depression_rates, color=sns.color_palette("RdYlBu_r", len(degrees)))
plt.xlabel('學歷', fontsize=12)
plt.ylabel('憂鬱症比例 (%)', fontsize=12)
plt.title('不同學歷憂鬱症比例', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. K-means聚類分析

In [ ]:
# 選擇特徵
features = ['Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction']
X = df[features]

# 標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-means聚類
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# 聚類結果統計
cluster_stats = df.groupby('Cluster')['Depression'].agg(['count', 'mean']).round(3)
cluster_stats.columns = ['樣本數', '憂鬱比例']
print("K-means聚類結果：")
print(cluster_stats)

# 視覺化
plt.figure(figsize=(8, 6))
plt.bar(cluster_stats.index, cluster_stats['憂鬱比例'], 
        color=['#3498db', '#e74c3c', '#f39c12'])
plt.xlabel('群組')
plt.ylabel('憂鬱比例')
plt.title('K-means聚類結果')
plt.show()

## 5. 邏輯回歸分析

In [ ]:
# 準備資料
X = df[features]
y = df['Depression']

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 訓練模型
lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)

# 預測
y_pred = lr.predict(X_test_scaled)
y_pred_proba = lr.predict_proba(X_test_scaled)[:, 1]

# 評估
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"模型準確率：{accuracy:.3f}")

In [ ]:
# 混淆矩陣
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('混淆矩陣')
plt.ylabel('實際')
plt.xlabel('預測')
plt.show()

# ROC曲線
plt.figure(figsize=(6, 5))
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, linewidth=2, label=f'ROC曲線 (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('假陽性率')
plt.ylabel('真陽性率')
plt.title('ROC曲線')
plt.legend()
plt.show()

## 6. 特徵重要性分析

In [ ]:
# 顯示特徵係數
coef_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': lr.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("特徵重要性：")
print(coef_df)

# 視覺化
plt.figure(figsize=(8, 5))
plt.barh(coef_df['Feature'], coef_df['Coefficient'], color='skyblue')
plt.xlabel('係數值')
plt.title('邏輯回歸特徵係數')
plt.tight_layout()
plt.show()

## 7. 結果總結

### 待填寫結果：
- 資料筆數：____
- 欄位數量：____
- 整體憂鬱症比例：____％
- 最高風險學歷：____（憂鬱率：____％）
- 最低風險學歷：____（憂鬱率：____％）
- K-means最高風險群組：Cluster ____（憂鬱率：____％）
- 邏輯回歸準確率：____％
- AUC值：____
- 最重要特徵：____